# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import gmaps
from scipy.stats import linregress
import scipy.stats as st

# gmaps API Keys
from config import (api_key)
#from config import (gkey)

In [2]:
# pip install citipy
from citipy import citipy

## Generate Cities List

In [3]:
cities_list = []
latitude_longitude_pairs = []
np.random.seed(250)  # to allow others to duplicate 
lats = np.random.randint(-90.00, 90.00, size=2000)
longs = np.random.randint(-180.00, 180.00, size=2000)
latitude_longitude_pairs = zip(lats, longs) 

city_table_df = pd.DataFrame(columns = ['Latitude', 'Longitude', 'City'])

for latitude_longitude_pair in latitude_longitude_pairs:
    returned_city = citipy.nearest_city(latitude_longitude_pair[0],latitude_longitude_pair[1]).city_name
    
    if returned_city not in cities_list:
       cities_list.append(returned_city) 
       city_table_df = city_table_df.append({'Latitude' : latitude_longitude_pair[0], 'Longitude' : latitude_longitude_pair[1], 'City' : returned_city}, ignore_index = True)  
        
#len(list_of_cities)  
city_table_df.head(20)

,Latitude,Longitude,City
0,20,-54,saint-francois
1,-13,156,gizo
2,-65,-173,vaini
3,-32,79,bambous virieux
4,-39,-143,mataura
5,66,-67,pangnirtung
6,-61,22,bredasdorp
7,-1,104,simpang
8,75,-11,husavik
9,22,-105,ruiz


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

query_url = f"{url}appid={api_key}&units={units}&q="

In [5]:
# set up lists to hold reponse info
lat = []
long = []
temp = []
humid = []
wind = []
speed = []
cloud = []
cities2 = []

# Loop through the list of cities and perform a request for data on each
for city in cities_list:
    response = requests.get(query_url + city).json()
    
    try:
        lat.append(response['coord']['lat'])
        long.append(response['coord']['lon'])
        temp.append(response['main']['temp'])
        humid.append(response['main']['humidity'])
        wind.append(response['wind']['speed'])
        cloud.append(response['clouds']['all'])
        cities2.append(response['name'])
   
    except:
        pass

print('this message') 

this message


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [6]:
weather_datasofar = {"city": cities2,
                     "latitude": lat,
                     "longitude": long,
                     "temperature": temp,
                     "humidity": humid,
                     "wind speed": wind,
                     "cloudiness": cloud}
weather_guide_df = pd.DataFrame(weather_datasofar)
weather_guide_df 

,city,latitude,longitude,temperature,humidity,wind speed,cloudiness
0,Saint-François,46.4154,3.9054,44.55,93,3.44,75
1,Gizo Government Station,-8.1030,156.8419,82.87,74,6.89,71
2,Vaini,-21.2000,-175.2000,77.00,88,13.80,40
3,Bambous Virieux,-20.3428,57.7575,80.33,74,12.66,75
4,Mataura,-46.1927,168.8643,54.00,76,5.01,53
...,...,...,...,...,...,...,...
666,Kafanchan,9.5833,8.3000,73.62,68,1.92,47
667,Bhasāwar,27.0333,77.0333,100.56,13,8.88,67
668,Znamenskoye,57.1281,73.8282,40.69,57,14.76,59
669,Atbasar,51.8000,68.3333,50.56,30,9.64,2


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [7]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [8]:
from config import (api_key, gkey)

gmaps.configure(api_key=gkey) 
#weather_guide.dtypes

In [9]:
locations = weather_guide_df[["latitude", "longitude"]].astype(float)

In [10]:
fig = gmaps.figure()

humidity_rate = weather_guide_df["humidity"].astype(float)
#humidity_rate.dtypes
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, dissipating=False, max_intensity=100, point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
humidity_numbers = weather_guide_df["humidity"].tolist()
humidity_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)', stroke_color='rgba(0, 0 , 150, 0.4)',
                                    scale=2, info_box_content=[f"Humidity: {humidity_number}" for humidity_number in 
                                                               humidity_numbers])

fig = gmaps.figure()
fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
#mouse_metadata_df.drop(mouse_metadata_df[mouse_metadata_df['Mouse ID']=='g989'].index, inplace=True)
weather_guide_df.drop(weather_guide_df[weather_guide_df['cloudiness']>0].index, inplace=True)
weather_guide_df.drop(weather_guide_df[weather_guide_df['wind speed']>10].index, inplace=True)
weather_guide_df.drop(weather_guide_df[weather_guide_df['temperature']<70].index, inplace=True)
weather_guide_df.drop(weather_guide_df[weather_guide_df['temperature']>80].index, inplace=True)

#len(weather_guide_df)
weather_guide_df.head(20)

,city,latitude,longitude,temperature,humidity,wind speed,cloudiness
22,Mahanoro,-19.9000,48.8000,76.15,78,3.58,0
30,Kiryat Gat,31.6100,34.7642,70.84,59,5.75,0
113,Sakakah,29.9697,40.2064,75.20,17,5.75,0
139,Touros,-5.1989,-35.4608,73.40,94,4.61,0
179,Lujiang,31.2581,117.2817,75.88,27,6.71,0
189,Le Vauclin,14.5452,-60.8388,73.17,83,4.61,0
258,Ambilobe,-13.2000,49.0500,78.51,81,3.06,0
313,Murgab,37.4966,61.9714,71.60,53,9.22,0
317,Bela Vista de Goiás,-16.9728,-48.9533,71.60,78,4.61,0
324,Mossamedes,-15.1961,12.1522,75.13,70,4.70,0


In [ ]:
# base url
import pprint
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in weather_guide_df.iterrows():
    
    target_coordinates = weather_guide_df['latitude'].map(str) + ',' + weather_guide_df['longitude'].map(str)
    #print (target_coordinates)
    params = {
    "location": target_coordinates,
    "radius": 5000,
    "types": "lodging",
    "key": gkey
         }
    #print (params)
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        #print(results[0]['name'])
        pprint.pprint(results)
        #weather_guide_df['Hotel Name'] = response['name']
    except:
        pass
    